# Klasifikasi

* Klasifikasi Menggunakan Dataset Asli + Dataset Hasil Peembangkitan Kata Spam tipe-3
* Meliputi Data Non Spam dan Data Spam Tipe-3

In [ ]:
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

# Fungsi Persiapan Data

In [ ]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

# Persiapan Data

* Baca Data
* Menghapus null Line
* Data yang digunakan : Data Asli Spam Tipe-3 + Data Hasil Pembangkitan kata + Data Non Spam

In [ ]:
# load cleaned original spam text 

linessel = pd.read_csv('Data/train_data - juli - Asli.csv', sep=',', encoding='latin-1')
lines_sel = linessel['Body']

# load cleaned text generated 
in_filename_gen = 'Data/Output/7. Generated/gen_w2v_lstm.txt'
doc_gen = load_doc(in_filename_gen)
lines_gen = doc_gen.split('\n')

# Load Non Spam text
in_filename_rev = 'Data/Output/8. Klasifikasi/notspam.txt'
doc_rev = load_doc(in_filename_rev)
lines_rev = doc_rev.split('\n')

for i in lines_sel:
    if i=="":
        lines_sel.remove(i)

for i in lines_gen:
    if i=="":
        lines_gen.remove(i)
        
for i in lines_rev:
    if i=="":
        lines_rev.remove(i)

# Ekstraksi Fitur

* Menggunakan pembobotan TfIdf

In [ ]:
vectorizertfidf1 = TfidfVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1)

corpus_sel=[text_preprocess(i) for i in lines_sel]
corpus_gen=[text_preprocess(i) for i in lines_gen]
corpus_rev=[text_preprocess(i) for i in lines_rev]

A = vectorizertfidf1.fit_transform(corpus_sel)
B = vectorizertfidf1.transform(corpus_gen)
C = vectorizertfidf1.transform(corpus_rev)

ngram_sel = pd.DataFrame(A.toarray())
ngram_gen = pd.DataFrame(B.toarray())
ngram_rev = pd.DataFrame(C.toarray())

fitur_sel=ngram_sel.fillna(0)
fitur_gen=ngram_gen.fillna(0)
fitur_rev=ngram_rev.fillna(0)

#fitur_sel.to_csv('Data/Output/5. Sequence/nfitur-sel.csv', index=False)
#fitur_gen.to_csv('Data/Output/5. Sequence/nfitur-gen.csv', index=False)
#fitur_rev.to_csv('Data/Output/5. Sequence/nfitur-rev.csv', index=False)

#penggaubngan data fitur tf idf
fitur=pd.DataFrame(A.toarray()) # tambahan
fitur=fitur.append(ngram_rev, ignore_index = True) #review
fitur=fitur.append(ngram_gen, ignore_index = True) # spam

# Traning Model Klasifikasi

In [ ]:
import time
start_time = time.time()

#labeling
y_l=[]
y_l=[1 for i in range(109)] #label spam 
for i in range(326):
    y_l.append(0) # label non spam
for i in range(109): # label spam generation - kontribusi
    y_l.append(1)

y = pd.DataFrame(y_l, columns=['Label'])
from sklearn import svm

#Modeling
from sklearn.model_selection import cross_validate

clf= svm.SVC(kernel='linear', C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=2, gamma='auto', max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

#Training dan Evaluasi
scores= cross_validate(clf, fitur, y, cv=10, scoring=['f1', 'recall'])

# Print Hasil Evaluasi 

In [ ]:
import statistics

print (statistics.mean(scores['test_f1']))
print(scores['test_f1'])
print(max(scores['test_f1']))
print(min(scores['test_f1']))

print (statistics.mean(scores['test_recall']))
print((scores['test_recall']))
print(max(scores['test_recall']))
print(min(scores['test_recall']))